# ECCOv4 access via NASA's EarthData in the Cloud

This notebook demonstrates access to [ECCOv4](https://ecco-group.org/) model output. Broad information about the ECCO dataset can be found in the PODAAC website (see [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO)).

**Requirements to run this notebook**
1. Have an Earth Data Login account
2. Have a Bearer Token.

**Objectives**
 
Use [pydap](https://pydap.github.io/pydap/)'s client API to demonstrate

- Access to NASA's [EarthData](https://www.earthdata.nasa.gov/) via the use of `tokens`. `Tokens` are greatly favored over `username/password`, since `tokens` avoid repeated authentication over the many redirects.
- A workflow involving multiple OPeNDAP URLs and xarray parallelism, lazy evaluation, and plotting of **Level 4** with complex Topology [ECCOv4](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO) Data via OPeNDAP.


Some variables of focus are

- [Native grid](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4)
- [Temperature and Salinity](https://podaac.jpl.nasa.gov/dataset/ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4)



`Author`: Miguel Jimenez-Urias, '24

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pydap.net import create_session
from pydap.client import open_url
import xarray as xr
import pydap

In [ ]:
print("xarray version: ", xr.__version__)
print("pydap version: ", pydap.__version__)

### Access EARTHDATA

Many of the data variables can be browsed [here](https://podaac.jpl.nasa.gov/cloud-datasets?view=list&ids=Projects&values=ECCO). Here we will work with the original data defined on the Lat-Lon-Cap (LLC90) grid.

In [ ]:
Grid_url = 'dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Geometry%20Parameters%20for%20the%20Lat-Lon-Cap%2090%20(llc90)%20Native%20Model%20Grid%20(Version%204%20Release%204)/granules/GRID_GEOMETRY_ECCO_V4r4_native_llc0090'

### Use `.netrc` credentials



In [ ]:
my_session = create_session()

### Alternative token approach
```python
session_extra = {"token": "YourToken"}

# initialize a requests.session object with the token headers. All handled by pydap.
my_session = create_session(session_kwargs=session_extra)
```

### Lazy access to remote data via pydap's client API

`pydap` exploits the OPeNDAP's separation between metadata and data, to create `lazy dataset objects that point to the data`. These lazy objects contain all the attributes detailed in OPeNDAP's metadata files (DMR).

In [ ]:
ds_grid = open_url(Grid_url, session=my_session)

In [ ]:
ds_grid.tree()

```{note}
PyDAP accesses the remote dataset's metadata, and no data has been downloaded yet!
```

**Download data into memory**

The syntax is as follows:

```python
# this fetches remote data into a pydap object container
pydap_Array = dataset[<VarName>][:]
```
where `<VarName>` is the name of one of the variables in the `pydap.model.BaseType` object. The `pydap.model.BaseType` is a thing wrapper of `numpy arrays`. The array has been downloaded into "local" memory (RAM) as (uncompressed) numpy arrays. 

To extract the data as a numpy array, run

```python
# The `.data` command allows direct access to the Numpy array (e.g. for manipulation)
pydap_Array.data
```



In [ ]:
# lets download some data
Depth = ds_grid['Depth'][:]
print(type(Depth))

In [ ]:
Depth.attributes

In [ ]:
Depth.shape, Depth.dims

**Plot Depth along native grid**

`ECCO` data is defined on a Cube Sphere -- meaning the horizontal grid contains an `extra` dimension: `tile` or `face`. You can inspect the data in its native grid by plotting all horizontal data onto a grid as follows:

In [ ]:
Variable = [Depth[i].data for i in range(13)]
clevels =  np.linspace(0, 6000, 100)
cMap = 'Greys_r'

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES = [
    axes[4, 0], axes[3, 0], axes[2, 0], axes[4, 1], axes[3, 1], axes[2, 1],
    axes[1, 1], 
    axes[1, 2], axes[1, 3], axes[1, 4], axes[0, 2], axes[0, 3], axes[0, 4],
]
for i in range(len(AXES)):
    AXES[i].contourf(Variable[i], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 1.** `Depth` plotted on a horizontal layout. Data on tiles `0-5` follow `C-ordering`, whereas data on tiles `7-13` follow `F-ordering`. Data on the `arctic cap`, is defined on a polar coordinate grid.

**Plot with corrected Topology**

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], clevels, cmap=cMap)

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], clevels, cmap=cMap)

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 2.** `Depth` plotted on a horizontal layout that approximates `lat-lon` display. Data on the `arctic cap`, however, remains on a polar coordinate grid.

### pydap + xarray to aggregate multiple URLs


`OPeNDAP` allows remote access via `dataURL`s. Each `dataURL` represents a variable, i.e. a piece of the whole puzzle. We can exploit `xarray` concatenation and parallelism to combine multiple `dataURL`s, and thus multiple pydap objects, into a single `xarray.Dataset`. Can do this because `xarray has long-implemented pydap as an engine to access/open remote datasets`.




**A single URL**

Many `OPeNDAP` servers can serve data in either `DAP2` and `DAP4` (much newer) model implementations. In `pydap`, you can specify which of the two implementations by replacing the beginning of the URL with either one of `dap2` or `dap4`. Since `xarray` imports pydap internally as is, then xarray can recognize this behavior as well.


Below we access remote Temperature and Salinity ECCO data with `xarray` via (internally) `pydap`.


In [ ]:
baseURL = 'dap4://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/'
Temp_Salt = "ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%20llc90%20Grid%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_"
year = '2017-'
month = '01'
end_ = '_ECCO_V4r4_native_llc0090'
Temp_2017 = baseURL + Temp_Salt +year  + month + end_


In [ ]:
Temp_2017

## Using CEs to pre-select only variables of interest

Allows the user to drop variables but ONLY after creating the dataset of the entire remote URL. With OPeNDAP servers, the you encode the names of the variables
you want to access in the URL. THis can significantly speed up analysis and exploration.

Below we manually specify the DAP4 Constraint Expression for keeping only the variables `THETA`, along with its dimensions `time`, `k`, `time`, `j`, `i` and `tile`. 


You can also construct the full URL with constraint expressions interactively, by selecting manually the variables on the datasets's [Data Request Form](https://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%20llc90%20Grid%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_2017-01_ECCO_V4r4_native_llc0090.dmr) and selecting **Copy (Raw) Data URL**.


In [ ]:
pyds = open_url(Temp_2017, session=my_session)
pyds.tree()

Pydap's object can now helps up construct the Constraint Expression that will allow us to create a much simpler datacube

In [ ]:
dims = pyds['THETA'].dims
Vars = ['/THETA'] + dims

# Below construct Contraint Expression
CE = "?dap4.ce="+(";").join(Vars)
print("Constraint Expression: ", CE)

Now we add the CE to the data url as follows below:

In [ ]:
Temp_2017 = baseURL + Temp_Salt +year  + month + end_ + CE
Temp_2017

In [ ]:
%%time
dataset = xr.open_dataset(Temp_2017, engine='pydap', session=my_session)
dataset

### Multiple files in parallel

We can exploit `xarray`'s intuitive `concat` + `merge` capabilities to read multiple data URL in parallel. To accomplish that, we need to create a list of all relevant URLs that each point to an OPeNDAP dataset.

```{tip}
In this case we will use a session that caches the response. It can handle the authentication too!
```

In [ ]:
Temp_2017 = [baseURL + Temp_Salt + year + f'{i:02}' + end_ + CE for i in range(1, 13)]

In [ ]:
Temp_2017[:4]

In [ ]:
cache_session = create_session(use_cache=True, cache_kwargs={'cache_name': 'ECCOv4'}) # caching the session
cache_session.cache.clear() # clear all cache to demonstrate the behavior. You do not have to do this!

```{warning}
For `Pydap >= 3.5.5` the new method `consolidated_metadata` can be used to download and cache all dmrs in parallel. This can significantly speed up the (xarray) dataset exploration. This method continues under active development and so its features will continue to evolve.
```



In [ ]:
from pydap.client import consolidate_metadata

In [ ]:
%%time
consolidate_metadata(Temp_2017, cache_session, concat_dim='time')

In [ ]:
%%time
theta_salt_ds = xr.open_mfdataset(
    Temp_2017, 
    engine='pydap',
    session=cache_session, 
    parallel=True,
    combine='nested',
    concat_dim='time',
)

In [ ]:
theta_salt_ds

### Finally, we plot `THETA`

In [ ]:
Variable = [theta_salt_ds['THETA'][0, 0, i, :, :] for i in range(13)]
clevels = np.linspace(-5, 30, 100)
cMap='RdBu_r'

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8), gridspec_kw={'hspace':0.01, 'wspace':0.01})
AXES_NR = [
    axes[3, 0], axes[2, 0], axes[1, 0], axes[3, 1], axes[2, 1], axes[1, 1],
]
AXES_CAP = [axes[0, 0]]
AXES_R = [
    axes[1, 2], axes[2, 2], axes[3, 2], axes[1, 3], axes[2, 3], axes[3, 3],
]
for i in range(len(AXES_NR)):
    AXES_NR[i].contourf(Variable[i], clevels, cmap=cMap)

for i in range(len(AXES_CAP)):
    AXES_CAP[i].contourf(Variable[6].transpose()[:, ::-1], clevels, cmap=cMap)

for i in range(len(AXES_R)):
    AXES_R[i].contourf(Variable[7+i].transpose()[::-1, :], clevels, cmap=cMap)

for ax in np.ravel(axes):
    ax.axis('off')
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)

plt.show()

**Fig. 3.** `Surface temperature`, plotted on a horizontal layout that approximates `lat-lon` display. Data on the `arctic cap`, however, remains on a polar coordinate grid.